In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
data = pd.read_csv('train.csv')
data= data.drop(['Outlet_Identifier'],1)
data= data.drop(['Item_Identifier'],1)
data['Outlet_Size'].fillna('Medium', inplace = True)
data = data[:150]
min_elements_in_leaf = 3
data2 = pd.DataFrame()
data2['Outlet_Size'] = data['Outlet_Size']
data2['Outlet_Type'] = data['Outlet_Type']
data2['Item_Outlet_Sales'] = data['Item_Outlet_Sales']
data = data2

for col in [ 'Outlet_Size','Outlet_Type']:
    data[col] = data[col].astype('category')

cat_col = []

for col in data:
    if data[col].dtype.name == 'category':
        cat_col.append(col)

result = 'Item_Outlet_Sales'
attrib_value_pair_used = {}

some_avail = {}
for col in data:
    some_avail[col] = 1


In [3]:
def categorical_split_err(dataset, feature):
    minvar = 99999999 

    print('now in categorical_split_err function with feature', feature)
    print 'no of elements is ',len(dataset.index)
    #attrib_val = 2 -------never used 
    used_val = 2
    #print('in categorical someavail is ',feature, some_avail[feature])

    
    for val in dataset[feature].unique():
            #print('in ourside of val ',val)
            if(  (feature, val) not in attrib_value_pair_used):
                #print('checking for val',val)
                #print('trying val', val)
                ds = dataset.loc[dataset[feature] == val]
                var = np.var(ds[result])
                #print('variance is ', var)
                if(var < minvar):
                    used_val  = val
                    minvar = var
    #if(feature == cat_col[0]):
    #print('returning error of', minvar, feature, 'value to be', used_val)
    
    
    return ({'error':minvar, 'attrib_value': used_val})

In [4]:
class myTree:
    def __init__(self, feature = None ,value =None,  left = None, right = None ):
        self.feature = feature
        self.value  = value
        self.left = left
        self.right = right

In [5]:
attrib_value_pair_used = {}
countCheck = 0
some_avail = {}

for col in data:
    some_avail[col] = 1


def buildTree(node,dataset ):
    #global x
    global countCheck
    global attrib_value_pair_used
    global some_avail
    #print('in build')
    
    mini = 9999999999 
    
    node.feature  = cat_col[0]
    node.value  = 7
    attrib_value = 5
    
    col_used = cat_col[0]
    changed = 0 
    
    for col in cat_col:

        
        if(some_avail[col] == 0):
            #print('someavail o for ',col)
            continue
        
        
        #print('avail is', some_avail[col], 'for ',col)
        
        recievedError = categorical_split_err(dataset, col) 
        errata = recievedError['error']
        attrival = recievedError['attrib_value']
        if( errata < mini):
                print 'it si less'
                mini = errata
                col_used = col
                attrib_value = attrival
                changed  = 1
        
        print('err mini is ',mini)
        
        node.feature =  col_used
        node.value =  attrib_value
        
    
    print('to be used is ',node.feature,'of value ', node.value)
    
    attrib_value_pair_used[(node.feature,node.value)] = 1
    
    
    somePresent = 0
    for valu in dataset[node.feature].unique():
        if((node.feature, valu) not in attrib_value_pair_used):
            somePresent =  1 
            #print('made avail bcoz in' ,node.feature, valu,' is present' )
            break
    if somePresent == 0:
        some_avail[node.feature] = 0 

    if(changed == 0):
        #pass 
        print('unchanged')
        return(0)
    #print('col is ',col)
    dataset_left = dataset.loc[dataset[node.feature] == attrib_value]
    dataset_right = dataset.loc[dataset[node.feature] != attrib_value]
    print('size dsataet left right ',len(dataset_left.index), len(dataset_right.index))
    print(attrib_value_pair_used)
    
    print('------------------------------')
    node.left = myTree()
    node.right = myTree()
    
           
    if (len(dataset_left.index) == 100 and len(dataset_right.index) == 100):
        print('cond true')
        return(0) 
    
    if(len(dataset_left.index)) > min_elements_in_leaf:
        print 'sending for left node'
        buildTree(node.left, dataset_left)
    else:
        print 'ans left is ',np.mean(dataset_left['Item_Outlet_Sales'])
        
    if(len(dataset_right.index) > min_elements_in_leaf):
        print 'sending for right node'
        buildTree(node.right,dataset_right)
    else:
        print 'ans right is ',np.mean(dataset_left['Item_Outlet_Sales'])
    
    
    

In [6]:
root = myTree()
buildTree(root, data)

('now in categorical_split_err function with feature', 'Outlet_Size')
no of elements is  150
it si less
('err mini is ', 872486.9644320676)
('now in categorical_split_err function with feature', 'Outlet_Type')
no of elements is  150
it si less
('err mini is ', 63184.702656260604)
('to be used is ', 'Outlet_Type', 'of value ', 'Grocery Store')
('size dsataet left right ', 19, 131)
{('Outlet_Type', 'Grocery Store'): 1}
------------------------------
sending for left node
('now in categorical_split_err function with feature', 'Outlet_Size')
no of elements is  19
it si less
('err mini is ', 32520.921041529884)
('now in categorical_split_err function with feature', 'Outlet_Type')
no of elements is  19
('err mini is ', 32520.921041529884)
('to be used is ', 'Outlet_Size', 'of value ', 'Medium')
('size dsataet left right ', 9, 10)
{('Outlet_Type', 'Grocery Store'): 1, ('Outlet_Size', 'Medium'): 1}
------------------------------
sending for left node
('now in categorical_split_err function wit

In [7]:
root.left.left

<__main__.myTree instance at 0x7fcc4fd88d88>

In [9]:
def printTree(node, name='Root', tabMultiplier = 0):
    if(node == None):
        return
    print("\t"*tabMultiplier),
    print(name, ": "),
    tempDict = {"feature" : node.feature, "value" : node.value}
    print(tempDict)
    tabMultiplier+=1
    printTree(node.left, 'Left', tabMultiplier)
    printTree(node.right, 'Right', tabMultiplier)

In [10]:
printTree(root)

 ('Root', ': ') {'feature': 'Outlet_Type', 'value': 'Grocery Store'}
	('Left', ': ') {'feature': 'Outlet_Size', 'value': 'Medium'}
		('Left', ': ') {'feature': 'Outlet_Size', 'value': 2}
			('Left', ': ') {'feature': None, 'value': None}
			('Right', ': ') {'feature': 'Outlet_Type', 'value': 2}
				('Left', ': ') {'feature': None, 'value': None}
				('Right', ': ') {'feature': 'Outlet_Size', 'value': 7}
		('Right', ': ') {'feature': 'Outlet_Size', 'value': 7}
	('Right', ': ') {'feature': 'Outlet_Size', 'value': 7}
